In [1]:
import openai
from openai import AzureOpenAI
import os
from pathlib import Path  
import json
import re
from dotenv import load_dotenv
env_path = Path('.') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
openai.api_key =  os.environ.get("AZURE_OPENAI_API_KEY")
openai.api_base =  os.environ.get("AZURE_OPENAI_ENDPOINT")
openai.api_type = "azure"
openai.api_version = "2023-12-01-preview"
deployment_id = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")
sql_server_username=os.environ.get("SQL_SERVER_USERNAME")
sql_server_password=os.environ.get("SQL_SERVER_PASSWORD")
sql_server_uri=os.environ.get("SQL_SERVER_URI")
sql_server_db=os.environ.get("SQL_SERVER_DB")
fabric_username=os.environ.get("AI_ASSISTANT_FABRIC_USER")
fabric_password=os.environ.get("AI_ASSISTANT_FABRIC_PASSWORD")
fabric_server=os.environ.get("FABRIC_SERVER")


In [14]:
client = AzureOpenAI(
  api_key=os.environ.get("AZURE_OPENAI_API_KEY"),  
  api_version="2023-12-01-preview",
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
)
messages= [
    {"role": "system", "content": """
     You are an AI assistant to help write SQL queries from natural languague. You follow the below workflow to answer the customer's question:
     1. Check the question customer asks has the same meaning with one of the answered questions below (remember it does not have to be exact same words, but just same meaning). If so, just return the SQL query you used to answer the question.
        Here is the list of historical answers:
        1. User question: What is the average number of orders per customer? Your answer: SELECT AVG(Orders.OrderID) FROM Orders
        2. User question: What is revenue per customer? Your answer: SELECT Customers.CustomerID, SUM(Orders.OrderID) FROM Customers INNER JOIN Orders ON Customers.CustomerID=Orders.CustomerID GROUP BY Customers.CustomerID

     2. If there question was not answered before, continue to the research steps below:
        - Use the get_tables() function to identify all tables within the data warehouse.
        - Determine which tables are pertinent to the customer's query.
        - Retrieve the schema of the relevant tables using the get_schema() function.
        - Sample data or execute a SQL query to understand the data within the relevant tables if needed using the execute_query() function.
        - Develop a SQL query tailored to the customer's needs by.
        - SQL query should be syntactically valid with SQL Server query syntax and rely on the schema of the discovered table(s)
        - Clarify the customer's question if necessary.
        - Return the SQL query to the customer.

     """},
    {"role": "user", "content": "Per customer, what is the revenue?"},
]



functions = [

    {
        "name": "get_schema",
        "description": "Retrieve detail schema of table",
        "parameters": {
            "type": "object",
            "properties": {
                "table_name": {
                    "type": "string",
                    "description": "name of the table to retrieve schema"
                }

            },
            "required": ["table_name"],
        },

    },
    {
        "name": "get_tables",
        "description": "Retrieve available tables in data warehouse",
        "parameters": {
            "type": "object",
            "properties": {
            },
            "required": [],
        },
    },
    {
        "name": "execute_query",
        "description": "execute a query on data warehouse",
        "parameters": {
            "type": "object",
            "properties": {
                "sql_query": {
                    "type": "string",
                    "description": "syntactically valid sql query that relies on the schema of the discovered table(s)"
                }

            },
            "required": ["sql_query"],
        },

    }


]
response = client.chat.completions.create(
    model=deployment_id, # model = "deployment_name"
    messages= messages,
    functions = functions,
    function_call="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))


{
  "content": null,
  "role": "assistant",
  "function_call": {
    "arguments": "{}",
    "name": "get_tables"
  },
  "tool_calls": null
}


In [3]:
client = AzureOpenAI(
  api_key=os.environ.get("AZURE_OPENAI_API_KEY"),  
  api_version="2023-12-01-preview",
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
)
messages= [
    {"role": "user", "content": """Find beachfront hotels in San Diego for less than $300 a month with free breakfast. 
     """}
]

functions= [  
    {
         "type": "function",
         "function":{
        "name": "search_hotels",
        "description": "Retrieves hotels from the search index based on the parameters provided",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location of the hotel (i.e. Seattle, WA)"
                },
                "max_price": {
                    "type": "number",
                    "description": "The maximum price for the hotel"
                },
                "features": {
                    "type": "string",
                    "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)"
                }
            },
            "required": ["location"]
        }
    }
    }
]  

response = client.chat.completions.create(
    model=deployment_id, # model = "deployment_name"
    messages= messages,
    tools = functions,
    tool_choice="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))
response_message = response["choices"][0]["message"]
tool_calls = response_message.tool_calls
def search_hotels():
    pass
# Check if the model wants to call a function
if tool_calls:

    # Call the function. The JSON response may not always be valid so make sure to handle errors
    function_name = response_message["function_call"]["name"]

    available_functions = {
            "search_hotels": search_hotels,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)

    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message["role"],
            "function_call": {
                "name": function_name,
                "arguments": response_message["function_call"]["arguments"],
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    ) 

    # Call the API again to get the final response from the model
    second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id="gpt-35-turbo-0613"
            # optionally, you could provide functions in the second call as well
        )
    print(second_response["choices"][0]["message"])
else:
    print(response["choices"][0]["message"])


{
  "content": null,
  "role": "assistant",
  "function_call": null,
  "tool_calls": [
    {
      "id": "call_CLO5AwgAr1pmpoqeqKlqXmTB",
      "function": {
        "arguments": "{\n  \"location\": \"San Diego, CA\",\n  \"max_price\": 300,\n  \"features\": \"beachfront, free breakfast\"\n}",
        "name": "search_hotels"
      },
      "type": "function"
    }
  ]
}


TypeError: 'ChatCompletion' object is not subscriptable

In [ ]:
response.choices[0].message.role

In [ ]:
import pyodbc
import csv

def execute_query(sql_query):
    # connection string
    conn_str = f"Driver={{ODBC Driver 17 for SQL Server}};Server={server},1433;Database=banking_lakehouse;UID={fabric_username};PWD={fabric_password};Authentication=ActiveDirectoryPassword;;Encrypt=yes;TrustServerCertificate=no;"

    # establish connection
    conn = pyodbc.connect(conn_str)

    # create cursor
    cursor = conn.cursor()

    # execute query
    cursor.execute(sql_query)

    # fetch all rows
    rows = cursor.fetchall()

    # convert rows to list of dictionaries
    result = [dict(zip([column[0] for column in cursor.description], row)) for row in rows]

    # convert result to CSV string
    csv_result = ""
    if result:
        csv_result = ",".join(result[0].keys()) + "\n"
        for row in result:
            csv_result += ",".join(str(value) for value in row.values()) + "\n"

    return csv_result

def get_table_names():
    # query to get the table names
    query = "SELECT name FROM customer_analytics.sys.tables"

    # execute query
    csv_result = execute_query(query)

    return csv_result

def get_table_schema(table_name):
    # query to get the table schema
    query = f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'"

    # execute query
    csv_result = execute_query(query)

    return csv_result

# Usage example:
table_names = get_table_names()
print(table_names)

# table_schema = get_table_schema('customer_contact')
# print(table_schema)
